In [1]:
import os
import time
import subprocess
import glob
from pprint import pprint
import numpy as np
from numpy.linalg import norm
import itertools 
import re
import pandas as pd
import shutil

root_dir = os.getcwd() + '\..'
sys.path.append(root_dir)

from orca_wrapper import run_orca, parse_orca_output, default_search_terms, get_imaginary_modes

os.chdir('Orca_Calculations/')

In [2]:
# Clean all directories
all_files = glob.glob(f'Step*/*')
non_input_files = []
for f in all_files:
    if f[-4:] == '.inp':
        #print(f)
        pass
    else:
        non_input_files.append(f)
non_input_files

['Step1\\Br-.gbw',
 'Step1\\Br-.hess',
 'Step1\\Br-.out',
 'Step1\\Br-.prop',
 'Step1\\Br-_property.txt',
 'Step1\\CH3Br.engrad',
 'Step1\\CH3Br.gbw',
 'Step1\\CH3Br.hess',
 'Step1\\CH3Br.opt',
 'Step1\\CH3Br.out',
 'Step1\\CH3Br.prop',
 'Step1\\CH3Br.xyz',
 'Step1\\CH3Br_property.txt',
 'Step1\\CH3Br_trj.xyz',
 'Step1\\CH3Cl.engrad',
 'Step1\\CH3Cl.gbw',
 'Step1\\CH3Cl.hess',
 'Step1\\CH3Cl.opt',
 'Step1\\CH3Cl.out',
 'Step1\\CH3Cl.prop',
 'Step1\\CH3Cl.xyz',
 'Step1\\CH3Cl_property.txt',
 'Step1\\CH3Cl_trj.xyz',
 'Step1\\Cl-.gbw',
 'Step1\\Cl-.hess',
 'Step1\\Cl-.out',
 'Step1\\Cl-.prop',
 'Step1\\Cl-_property.txt',
 'Step2\\CH3Cl_Br-.engrad',
 'Step2\\CH3Cl_Br-.gbw',
 'Step2\\CH3Cl_Br-.hess',
 'Step2\\CH3Cl_Br-.opt',
 'Step2\\CH3Cl_Br-.out',
 'Step2\\CH3Cl_Br-.prop',
 'Step2\\CH3Cl_Br-.xyz',
 'Step2\\CH3Cl_Br-_init.xyz',
 'Step2\\CH3Cl_Br-_property.txt',
 'Step2\\CH3Cl_Br-_trj.xyz',
 'Step2\\Cl-_CH3Br.engrad',
 'Step2\\Cl-_CH3Br.gbw',
 'Step2\\Cl-_CH3Br.hess',
 'Step2\\Cl-_CH3Br.opt

In [3]:
#for f in non_input_files: os.remove(f)

# Step 1 - Optimize Non-interacting Geometries

In [4]:
step_dir = 'Step1/'
input_files = glob.glob(f'{step_dir}/*.inp')
output_files = [i_file.replace('.inp', '.out') for i_file in input_files]
output_files

['Step1\\Br-.out', 'Step1\\CH3Br.out', 'Step1\\CH3Cl.out', 'Step1\\Cl-.out']

In [5]:
for i_file in input_files:
    run_orca(i_file)


Running "Step1\Br-.inp"
Writing to "Step1\Br-.out"

Running "Step1\CH3Br.inp"
Writing to "Step1\CH3Br.out"

Running "Step1\CH3Cl.inp"
Writing to "Step1\CH3Cl.out"

Running "Step1\Cl-.inp"
Writing to "Step1\Cl-.out"


# Step 2 - Optimize Weakly Interacting Geometries

In [6]:
# Use the optimized geometry for CH3Br as a starting point for our sweep
with open('Step1/CH3Br.xyz') as init_xyz:
    lines = init_xyz.readlines()
    nbr_atoms_before = lines[0].strip()
    nbr_atoms_after = 6
    lines[0] = f'{nbr_atoms_after}\n'
    lines.append('  Cl  -3.7                  0.0                    0.0\n')

for line in lines:
    #print(line, end='')
    pass

with open('Step2/Cl-_CH3Br_init.xyz', 'w') as new_xyz:
    new_xyz.write(''.join(lines))

# Use the optimized geometry for CH3Cl as a starting point for our sweep
with open('Step1/CH3Cl.xyz') as init_xyz:
    lines = init_xyz.readlines()
    nbr_atoms_before = lines[0].strip()
    nbr_atoms_after = 6
    lines[0] = f'{nbr_atoms_after}\n'
    lines.append('  Br  -3.7                  0.0                    0.0\n')

for line in lines:
    #print(line, end='')
    pass

with open('Step2/CH3Cl_Br-_init.xyz', 'w') as new_xyz:
    new_xyz.write(''.join(lines))

In [7]:
input_files = glob.glob(f'Step2/*.inp')
output_files = [i_file.replace('.inp', '.out') for i_file in input_files]
output_files

['Step2\\CH3Cl_Br-.out', 'Step2\\Cl-_CH3Br.out']

In [8]:
for i_file in input_files:
    run_orca(i_file)


Running "Step2\CH3Cl_Br-.inp"
Writing to "Step2\CH3Cl_Br-.out"

Running "Step2\Cl-_CH3Br.inp"
Writing to "Step2\Cl-_CH3Br.out"


# Step 3 - Scan for Transition State

In [9]:
shutil.copy('Step2/Cl-_CH3Br.xyz', 'Step3/Cl-_CH3Br_init.xyz')

'Step3/Cl-_CH3Br_init.xyz'

In [10]:
step_dir = 'Step3'
input_files = glob.glob(f'{step_dir}/*.inp')
output_files = [i_file.replace('.inp', '.out') for i_file in input_files]


In [11]:
for i_file in input_files:
    run_orca(i_file)


Running "Step3\Cl-_CH3Br.inp"
Writing to "Step3\Cl-_CH3Br.out"


In [12]:
for o_file in output_files:
    # get the highest energy transition state
    with open(o_file, 'r') as of:
        content = of.read()
        split_str = "The Calculated Surface using the 'Actual Energy'\n"
        trans_state_values = content.split(split_str)[1].split('\n\n',1)[0].split('\n')
    print(trans_state_values)
    energies = [float(s.strip().split()[1]) for s in trans_state_values ]

pprint(energies)

ts_index = np.argmax(energies) + 1
print(f'\nTransition state supected at index {ts_index}')

['   2.90000000 -3073.63538623', '   2.70000000 -3073.63446451', '   2.50000000 -3073.63260691', '   2.30000000 -3073.63602085', '   2.10000000 -3073.64416383', '   1.90000000 -3073.65156792']
[-3073.63538623,
 -3073.63446451,
 -3073.63260691,
 -3073.63602085,
 -3073.64416383,
 -3073.65156792]

Transition state supected at index 3


# Step 4 - Find Imaginary Modes

In [13]:
shutil.copy(f'Step3/Cl-_CH3Br.{ts_index:03}.xyz', 'Step4/Cl-_CH3Br_TS_init.xyz')

'Step4/Cl-_CH3Br_TS_init.xyz'

In [14]:
input_files = glob.glob(f'Step4/*.inp')
output_files = [i_file.replace('.inp', '.out') for i_file in input_files]
output_files

['Step4\\[Cl_CH3_Br]-.out']

In [15]:
for i_file in input_files:
    run_orca(i_file)


Running "Step4\[Cl_CH3_Br]-.inp"
Writing to "Step4\[Cl_CH3_Br]-.out"


In [16]:
im_freq_lines = get_imaginary_modes(output_files[0])
im_freqs = []
im_freq_idx = []
for line in im_freq_lines:
    idx_str, freq_str, _ = line.split(None, 2)
    idx = int(idx_str[:-1])
    freq = float(freq_str)
    if idx not in im_freq_idx:
        im_freq_idx.append(idx)
        im_freqs.append(freq)

if len(im_freq_idx) > 1:
    print("More than one imginary frequency found! This is not a transition state!")
    print(im_freq_lines)
elif len(im_freq_idx) < 1:
    print("No imgaginary frequencies found!")
else:
    reaction_idx = im_freq_idx[0]
    print(f"Reaction coordinate given by mode {reaction_idx} with imaginary frequency {im_freqs[0]} cm**-1")

Reaction coordinate given by mode 6 with imaginary frequency -282.42 cm**-1


In [17]:
hess_file = [i_file.replace('.inp', '.hess') for i_file in input_files]
print(hess_file[0])
p_out = subprocess.run(['orca_pltvib', hess_file[0], str(reaction_idx)], capture_output=True)
if p_out.stderr:
    print(p_out.stderr.decode())
print(p_out.stdout.decode())

Step4\[Cl_CH3_Br]-.hess
The file is being recognized as a valid orca hessian file
Will plot 1 vibrations:
   6
creating: Step4\[Cl_CH3_Br]-.hess.v006.xyz



# Step 5 - Single Point Calculations with Bigger Basis

In [18]:
input_files = glob.glob('Step5/*.inp')
output_files = [i_file.replace('.inp', '.out') for i_file in input_files]
output_files

['Step5\\Br-.out',
 'Step5\\CH3Br.out',
 'Step5\\CH3Cl.out',
 'Step5\\CH3Cl_Br-.out',
 'Step5\\Cl-.out',
 'Step5\\Cl-_CH3Br.out',
 'Step5\\[Cl_CH3_Br]-.out']

In [19]:
for i_file in input_files:
    run_orca(i_file)


Running "Step5\Br-.inp"
Writing to "Step5\Br-.out"

Running "Step5\CH3Br.inp"
Writing to "Step5\CH3Br.out"

Running "Step5\CH3Cl.inp"
Writing to "Step5\CH3Cl.out"

Running "Step5\CH3Cl_Br-.inp"
Writing to "Step5\CH3Cl_Br-.out"

Running "Step5\Cl-.inp"
Writing to "Step5\Cl-.out"

Running "Step5\Cl-_CH3Br.inp"
Writing to "Step5\Cl-_CH3Br.out"

Running "Step5\[Cl_CH3_Br]-.inp"
Writing to "Step5\[Cl_CH3_Br]-.out"


# Step 6 - Solvation Effects

In [20]:
input_files = glob.glob('Step6/*.inp')
output_files = [i_file.replace('.inp', '.out') for i_file in input_files]
output_files

['Step6\\Br-.out',
 'Step6\\CH3Br.out',
 'Step6\\CH3Cl.out',
 'Step6\\CH3Cl_Br-.out',
 'Step6\\Cl-.out',
 'Step6\\Cl-_CH3Br.out',
 'Step6\\[Cl_CH3_Br]-.out']

In [21]:
for i_file in input_files:
    run_orca(i_file)


Running "Step6\Br-.inp"
Writing to "Step6\Br-.out"

Running "Step6\CH3Br.inp"
Writing to "Step6\CH3Br.out"

Running "Step6\CH3Cl.inp"
Writing to "Step6\CH3Cl.out"

Running "Step6\CH3Cl_Br-.inp"
Writing to "Step6\CH3Cl_Br-.out"

Running "Step6\Cl-.inp"
Writing to "Step6\Cl-.out"

Running "Step6\Cl-_CH3Br.inp"
Writing to "Step6\Cl-_CH3Br.out"

Running "Step6\[Cl_CH3_Br]-.inp"
Writing to "Step6\[Cl_CH3_Br]-.out"


# Putting Together the Results

In [46]:
output_files = glob.glob('Step*/*.out')
output_files

['Step1\\Br-.out',
 'Step1\\CH3Br.out',
 'Step1\\CH3Cl.out',
 'Step1\\Cl-.out',
 'Step2\\CH3Cl_Br-.out',
 'Step2\\Cl-_CH3Br.out',
 'Step3\\Cl-_CH3Br.out',
 'Step4\\[Cl_CH3_Br]-.out',
 'Step5\\Br-.out',
 'Step5\\CH3Br.out',
 'Step5\\CH3Cl.out',
 'Step5\\CH3Cl_Br-.out',
 'Step5\\Cl-.out',
 'Step5\\Cl-_CH3Br.out',
 'Step5\\[Cl_CH3_Br]-.out',
 'Step6\\Br-.out',
 'Step6\\CH3Br.out',
 'Step6\\CH3Cl.out',
 'Step6\\CH3Cl_Br-.out',
 'Step6\\Cl-.out',
 'Step6\\Cl-_CH3Br.out',
 'Step6\\[Cl_CH3_Br]-.out']

In [47]:
calcs = {}
for o_file in output_files:
    terms = parse_orca_output(o_file)
    if 'G' not in terms:
        print(f'\nNo entropy in "{o_file}"')
        search_terms = {'TS_trans': 'Translational entropy'}
        trans_entr = parse_orca_output(o_file, search_terms=search_terms)
        if 'TS_trans' in trans_entr:
            print(f'Found translational entropy, calculating Gibbs energy')
            terms['G'] = terms['H'] - trans_entr['TS_trans']
            terms['G-E(el)'] = terms['G'] - terms['E_sp']
        else:
            print('Cannot calculate Gibbs energy.')

    else:
        print(f'\nEntropy found in "{o_file}"')


    calcs[o_file[:-4]] = terms # Use filename without file ending


raw_res_df = pd.DataFrame(calcs)
raw_res_df.T


No entropy in "Step1\Br-.out"
Found translational entropy, calculating Gibbs energy

Entropy found in "Step1\CH3Br.out"

Entropy found in "Step1\CH3Cl.out"

No entropy in "Step1\Cl-.out"
Found translational entropy, calculating Gibbs energy

Entropy found in "Step2\CH3Cl_Br-.out"

Entropy found in "Step2\Cl-_CH3Br.out"

No entropy in "Step3\Cl-_CH3Br.out"
Cannot calculate Gibbs energy.

Entropy found in "Step4\[Cl_CH3_Br]-.out"

No entropy in "Step5\Br-.out"
Cannot calculate Gibbs energy.

No entropy in "Step5\CH3Br.out"
Cannot calculate Gibbs energy.

No entropy in "Step5\CH3Cl.out"
Cannot calculate Gibbs energy.

No entropy in "Step5\CH3Cl_Br-.out"
Cannot calculate Gibbs energy.

No entropy in "Step5\Cl-.out"
Cannot calculate Gibbs energy.

No entropy in "Step5\Cl-_CH3Br.out"
Cannot calculate Gibbs energy.

No entropy in "Step5\[Cl_CH3_Br]-.out"
Cannot calculate Gibbs energy.

No entropy in "Step6\Br-.out"
Cannot calculate Gibbs energy.

No entropy in "Step6\CH3Br.out"
Cannot calcul

,E_sp,E_therm,H,G,G-E(el),TS
Step1\Br-,-2573.783655,-2573.779406,-2573.778462,-2573.797015,-0.013360,NaN
Step1\CH3Br,-2613.579210,-2613.539257,-2613.538313,-2613.567287,0.011923,0.028975
Step1\CH3Cl,-499.848502,-499.808073,-499.807129,-499.834794,0.013708,0.027665
Step1\Cl-,-460.023549,-460.019300,-460.018356,-460.035759,-0.012210,NaN
Step2\CH3Cl_Br-,-3073.652284,-3073.609444,-3073.608500,-3073.645075,0.007209,0.036575
Step2\Cl-_CH3Br,-3073.635520,-3073.593542,-3073.592597,-3073.629046,0.006473,0.036449
Step3\Cl-_CH3Br,-3073.651568,NaN,NaN,NaN,NaN,NaN
Step4\[Cl_CH3_Br]-,-3073.632714,-3073.591893,-3073.590949,-3073.626053,0.006661,0.035104
Step5\Br-,-2574.129170,NaN,NaN,NaN,NaN,NaN
Step5\CH3Br,-2613.942478,NaN,NaN,NaN,NaN,NaN


In [48]:
step1_corr = ['Br-', 'Cl-', 'CH3Br', 'CH3Cl'] 
step2_corr = ['CH3Cl_Br-', 'Cl-_CH3Br'] 
step4_corr = ['[Cl_CH3_Br]-']

for stuff in step1_corr:
    raw_res_df[f'Step5\\{stuff}']['G'] = raw_res_df[f'Step5\\{stuff}']['E_sp'] + raw_res_df[f'Step1\\{stuff}']['G-E(el)']
    raw_res_df[f'Step6\\{stuff}']['G'] = raw_res_df[f'Step6\\{stuff}']['E_sp'] + raw_res_df[f'Step1\\{stuff}']['G-E(el)']
for stuff in step2_corr:
    raw_res_df[f'Step5\\{stuff}']['G'] = raw_res_df[f'Step5\\{stuff}']['E_sp'] + raw_res_df[f'Step2\\{stuff}']['G-E(el)']
    raw_res_df[f'Step6\\{stuff}']['G'] = raw_res_df[f'Step6\\{stuff}']['E_sp'] + raw_res_df[f'Step2\\{stuff}']['G-E(el)']
for stuff in step4_corr:
    raw_res_df[f'Step5\\{stuff}']['G'] = raw_res_df[f'Step5\\{stuff}']['E_sp'] + raw_res_df[f'Step4\\{stuff}']['G-E(el)']
    raw_res_df[f'Step6\\{stuff}']['G'] = raw_res_df[f'Step6\\{stuff}']['E_sp'] + raw_res_df[f'Step4\\{stuff}']['G-E(el)']

raw_res_df.T

,E_sp,E_therm,H,G,G-E(el),TS
Step1\Br-,-2573.783655,-2573.779406,-2573.778462,-2573.797015,-0.013360,NaN
Step1\CH3Br,-2613.579210,-2613.539257,-2613.538313,-2613.567287,0.011923,0.028975
Step1\CH3Cl,-499.848502,-499.808073,-499.807129,-499.834794,0.013708,0.027665
Step1\Cl-,-460.023549,-460.019300,-460.018356,-460.035759,-0.012210,NaN
Step2\CH3Cl_Br-,-3073.652284,-3073.609444,-3073.608500,-3073.645075,0.007209,0.036575
Step2\Cl-_CH3Br,-3073.635520,-3073.593542,-3073.592597,-3073.629046,0.006473,0.036449
Step3\Cl-_CH3Br,-3073.651568,NaN,NaN,NaN,NaN,NaN
Step4\[Cl_CH3_Br]-,-3073.632714,-3073.591893,-3073.590949,-3073.626053,0.006661,0.035104
Step5\Br-,-2574.129170,NaN,NaN,-2574.142530,NaN,NaN
Step5\CH3Br,-2613.942478,NaN,NaN,-2613.930555,NaN,NaN


In [49]:
res_df = raw_res_df.filter(regex='Step[1,2,4,5,6]') * 627.5094740631
res_df.T

,E_sp,E_therm,H,G,G-E(el),TS
Step1\Br-,-1.615074e+06,-1.615071e+06,-1.615070e+06,-1.615082e+06,-8.383735,NaN
Step1\CH3Br,-1.640046e+06,-1.640021e+06,-1.640020e+06,-1.640038e+06,7.481482,18.181886
Step1\CH3Cl,-3.136597e+05,-3.136343e+05,-3.136337e+05,-3.136511e+05,8.601850,17.359912
Step1\Cl-,-2.886691e+05,-2.886665e+05,-2.886659e+05,-2.886768e+05,-7.661584,NaN
Step2\CH3Cl_Br-,-1.928746e+06,-1.928719e+06,-1.928718e+06,-1.928741e+06,4.523961,22.950946
Step2\Cl-_CH3Br,-1.928735e+06,-1.928709e+06,-1.928708e+06,-1.928731e+06,4.062082,22.871923
Step4\[Cl_CH3_Br]-,-1.928734e+06,-1.928708e+06,-1.928707e+06,-1.928729e+06,4.180079,22.027948
Step5\Br-,-1.615290e+06,NaN,NaN,-1.615299e+06,NaN,NaN
Step5\CH3Br,-1.640274e+06,NaN,NaN,-1.640266e+06,NaN,NaN
Step5\CH3Cl,-3.137927e+05,NaN,NaN,-3.137841e+05,NaN,NaN


In [50]:
res_df['Step1\\Cl- + CH3Br'] = res_df['Step1\Cl-'] + res_df['Step1\CH3Br']
res_df['Step1\\CH3Cl + Br-'] = res_df['Step1\Br-'] + res_df['Step1\CH3Cl']
res_df['Step5\\Cl- + CH3Br'] = res_df['Step5\Cl-'] + res_df['Step5\CH3Br']
res_df['Step5\\CH3Cl + Br-'] = res_df['Step5\Br-'] + res_df['Step5\CH3Cl']
res_df['Step6\\Cl- + CH3Br'] = res_df['Step6\Cl-'] + res_df['Step6\CH3Br']
res_df['Step6\\CH3Cl + Br-'] = res_df['Step6\Br-'] + res_df['Step6\CH3Cl']
res_df.T

,E_sp,E_therm,H,G,G-E(el),TS
Step1\Br-,-1.615074e+06,-1.615071e+06,-1.615070e+06,-1.615082e+06,-8.383735,NaN
Step1\CH3Br,-1.640046e+06,-1.640021e+06,-1.640020e+06,-1.640038e+06,7.481482,18.181886
Step1\CH3Cl,-3.136597e+05,-3.136343e+05,-3.136337e+05,-3.136511e+05,8.601850,17.359912
Step1\Cl-,-2.886691e+05,-2.886665e+05,-2.886659e+05,-2.886768e+05,-7.661584,NaN
Step2\CH3Cl_Br-,-1.928746e+06,-1.928719e+06,-1.928718e+06,-1.928741e+06,4.523961,22.950946
Step2\Cl-_CH3Br,-1.928735e+06,-1.928709e+06,-1.928708e+06,-1.928731e+06,4.062082,22.871923
Step4\[Cl_CH3_Br]-,-1.928734e+06,-1.928708e+06,-1.928707e+06,-1.928729e+06,4.180079,22.027948
Step5\Br-,-1.615290e+06,NaN,NaN,-1.615299e+06,NaN,NaN
Step5\CH3Br,-1.640274e+06,NaN,NaN,-1.640266e+06,NaN,NaN
Step5\CH3Cl,-3.137927e+05,NaN,NaN,-3.137841e+05,NaN,NaN


We see that the the lowest energy state before the transition state is $Cl^-\;-\; CH_3Br$ with -3073.611844 Ha

In [56]:
svp_ref = res_df['Step2\\Cl-_CH3Br']
tzvpd_ref = res_df['Step5\\Cl-_CH3Br']
cpcmdmf_ref = res_df['Step6\\Cl- + CH3Br']

In [57]:
(res_df.filter(regex=r'Step[1,2,4]\\[\[]?[A-Za-z0-9-]*\S.[_+]+').T - svp_ref)['G'].T

Step2\CH3Cl_Br-      -10.057923
Step2\Cl-_CH3Br        0.000000
Step4\[Cl_CH3_Br]-     1.878525
Step1\Cl- + CH3Br     16.315366
Step1\CH3Cl + Br-     -1.733853
Name: G, dtype: float64

In [58]:
(res_df.filter(regex=r'Step[5]\\[\[]?[A-Za-z0-9-]*\S.[_+]+').T - tzvpd_ref)['G'].T

Step5\CH3Cl_Br-      -6.270190
Step5\Cl-_CH3Br       0.000000
Step5\[Cl_CH3_Br]-    4.270363
Step5\Cl- + CH3Br     5.232591
Step5\CH3Cl + Br-    -2.424877
Name: G, dtype: float64

In [59]:
(res_df.filter(regex=r'Step[6]\\[\[]?[A-Za-z0-9-]*\S.[_+]+').T - cpcmdmf_ref)['G'].T

Step6\CH3Cl_Br-        3.597008
Step6\Cl-_CH3Br        9.394677
Step6\[Cl_CH3_Br]-    18.802146
Step6\Cl- + CH3Br      0.000000
Step6\CH3Cl + Br-     -3.078803
Name: G, dtype: float64

In [60]:
# 1 kcal/mol = 4184 J/mol
def eyring(dG, T=297, unit='J/mol'):
    if unit == 'J/mol':
        pass
    elif unit == 'kcal/mol':
        dG *= 4184
    else:
        print(f"Unknown unit: {unit}")
        return None
    kappa = 1 # according to the instructions
    kB = 1.380649e-23 # J/K
    planck = 6.62607015e-34 # Js
    R = 8.31446261815324 # J/K/mol
    k = kappa * (kB * T / planck) * np.exp(-dG / (R * T)) # This should then have the unit 1/s
    return k

eyring(18.802146, unit='kcal/mol')

0.09039619125396903

In [32]:
table = res_df.T[['E_sp', 'H', 'G', 'G-E(el)']].sort_index().to_latex(na_rep='-', float_format='{:0.2f}'.format, escape=False)

lines = table.split('\n')
for i, line in enumerate(lines):
    pattern = re.compile(r'Step[0-9]+\\(?:\[?[A-Za-z0-9-]+[\s+\s]?[_]?\]?)*')#\\')
    matches = pattern.findall(line)
    if len(matches) == 1:
        mol = matches[0].split('\\')[1]
        tex_mol = f"\ce{{{mol}}}"
        new_line = line.replace(matches[0], tex_mol)
        new_line = new_line.replace(' nan &', ' - &')
        lines[i] = new_line


table = '\n'.join(lines)

print(table)

\begin{tabular}{lrrrr}
\toprule
{} &        E_sp &           H &           G &  G-E(el) \\
\midrule
\ce{Br- }         & -1615073.63 & -1615070.37 & -1615082.01 &    -8.38 \\
\ce{CH3Br }       & -1640045.72 & -1640020.05 & -1640038.23 &     7.48 \\
\ce{CH3Cl }       &  -313659.67 &  -313633.71 &  -313651.07 &     8.60 \\
\ce{CH3Cl }+ Br-  & -1928733.30 & -1928704.08 & -1928733.08 &     0.22 \\
\ce{Cl- }         &  -288669.14 &  -288665.88 &  -288676.80 &    -7.66 \\
\ce{Cl- }+ CH3Br  & -1928714.85 & -1928685.93 & -1928715.03 &    -0.18 \\
\ce{CH3Cl_Br- }   & -1928745.93 & -1928718.45 & -1928741.40 &     4.52 \\
\ce{Cl-_CH3Br }   & -1928735.41 & -1928708.47 & -1928731.35 &     4.06 \\
\ce{[Cl_CH3_Br]- }& -1928733.65 & -1928707.44 & -1928729.47 &     4.18 \\
\ce{Br- }         & -1615290.44 &         - & -1615282.06 &      nan \\
\ce{CH3Br }       & -1640273.67 &         - & -1640281.15 &      nan \\
\ce{CH3Cl }       &  -313792.68 &         - &  -313801.28 &      nan \\
\ce{CH3Cl }+ Br-  